In [1]:
cd ..

/home/user/sic-classification-utils


In [2]:
from industrial_classification.hierarchy.sic_hierarchy import SIC, SicCode, SicNode

from src.industrial_classification_utils.llm.llm import ClassificationLLM

INFO:industrial_classification_utils.utils.sic_data_access:Loading text from /home/user/sic-classification-utils/src/industrial_classification_utils/data/example/sic_2d_condensed.txt


In [3]:
nodes = [
        SicNode(sic_code=SicCode("A0111x"), description="Bird watching"),
        SicNode(sic_code=SicCode("A0112x"), description="Petting animals"),
]
lookup = {}
for node in nodes:
    lookup[str(node.sic_code)] = node
    lookup[node.sic_code.alpha_code] = node
    lookup[node.sic_code.alpha_code.replace("x", "")] = node
    if node.sic_code.n_digits > 1:
        lookup[node.sic_code.alpha_code[1:].replace("x", "")] = node

    if node.sic_code.n_digits == 4 and not node.children:  # noqa: PLR2004
        key = node.sic_code.alpha_code[1:5] + "0"
        lookup[key] = node
sic = SIC(nodes=nodes, code_lookup=lookup)

In [4]:
llm_class = ClassificationLLM(model_name="gemini-1.5-flash")

model_name: gemini-1.5-flash


In [5]:
code = llm_class.get_sic_code(industry_descr="school", job_description="teach maths", job_title="teacher")

In [6]:
code

SicResponse(codable=True, followup=None, sic_code='85100', sic_descriptive='Education', sic_candidates=[SicCandidate(sic_code='85100', sic_descriptive='Education', likelihood=1.0)], reasoning="The respondent's company's main activity is described as 'school', and the job title and description are 'teacher' and 'teach maths', respectively.  This clearly falls under the 'Education' category (SIC 85).  The specific four-digit code may vary depending on the type of school (e.g., primary, secondary, special needs), but the two-digit code is definitively 85.")

In [7]:
llm_class.sic = sic

In [8]:
llm_class.sic.nodes[0].sic_code

SicCode("A0111x")

In [9]:
prompt_candidate = llm_class._prompt_candidate(code="01120", activities=["giving belly rubs"])

In [10]:
all(x in prompt_candidate  for x in ["Code", "01120"])

True

In [11]:
prompt_candidate

'{Code: 01120, Title: Petting animals, Example activities: giving belly rubs}'

In [15]:
llm_class.sic.all_leaf_text()

,code,text
0,01.11,Bird watching
1,01.12,Petting animals
